# WMT17

* To English
* Segment level data
* Scores
* Pearson correlation

In [1]:
!pip install datasets
!pip install bert_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 7.3 MB/s 
     |████████████████████████████████| 141 kB 70.7 MB/s 
     |████████████████████████████████| 115 kB 65.9 MB/s 
     |████████████████████████████████| 101 kB 14.1 MB/s 
     |████████████████████████████████| 212 kB 62.2 MB/s 
     |████████████████████████████████| 596 kB 59.4 MB/s 
     |████████████████████████████████| 127 kB 71.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 60 kB 3.2 MB/s 
     |███████████████████████████

In [2]:
!git clone https://github.com/drehero/geneval

Cloning into 'geneval'...
remote: Enumerating objects: 229, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 229 (delta 10), reused 20 (delta 7), pack-reused 205
Receiving objects: 100% (229/229), 203.77 KiB | 8.49 MiB/s, done.
Resolving deltas: 100% (84/84), done.


In [3]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [4]:
import pathlib

import datasets
import numpy as np
import pandas as pd
from scipy.stats import pearsonr

from geneval.geneval.data.wmt import WMT17

## BERTScore


In [5]:
class config:
    metric_name = "bertscore"
    metric_path = "bertscore"
    model_type = "bert-base-uncased"

    out_path = pathlib.Path("/content/drive/MyDrive/results/wmt17")

    target_lang = "en"
    lang_pairs = ["cs-en", "de-en", "fi-en", "lv-en", "ru-en", "tr-en", "zh-en"]

In [6]:
scorer = datasets.load_metric(config.metric_path)

In [7]:
for lang_pair in config.lang_pairs:
    # load data
    wmt = WMT17(lang_pair)
    # compute score
    scores = scorer.compute(
        predictions=wmt.translations,
        references=wmt.references,
        lang=config.target_lang,
        model_type=config.model_type
    )
    # save
    df = pd.DataFrame({
        "translation": wmt.translations,
        "references": wmt.references,
        "source": wmt.sources,
        "human": wmt.scores,
        "metric_precision": scores["precision"],
        "metric_recall": scores["recall"],
        "metric_f1": scores["f1"],
    })
    fn = f"{lang_pair}-{config.model_type}.csv"
    df.to_csv(config.out_path / config.metric_name / fn, index=False)

In [9]:
# load scores and compute pearson correlation
results = {}
for lang_pair in config.lang_pairs:
    fn = f"{lang_pair}-{config.model_type}.csv"
    df = pd.read_csv(config.out_path / config.metric_name / fn)
    corr = pearsonr(df["metric_f1"], df["human"])[0]
    results[lang_pair] = corr

In [10]:
results

{'cs-en': 0.6432113791324501,
 'de-en': 0.6649739645604085,
 'fi-en': 0.8091542301807512,
 'lv-en': 0.6861870972050228,
 'ru-en': 0.6984675117977112,
 'tr-en': 0.7271820800105004,
 'zh-en': 0.7033989523217288}